In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
from surprise import Reader, Dataset, SVD
from surprise.model_selection.validation import cross_validate
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [2]:
links = pd.read_csv('dataset/links_small.csv')
links = links[links['tmdbId'].notnull()]['tmdbId'].astype('int')

metadata = pd.read_csv('dataset/movies_metadata.csv')
metadata['genres'] = metadata['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

/Users/hoyeonjang/PycharmProjects/flaskProject/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
s = metadata.apply(lambda x: pd.Series(x['genres']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
metadata = metadata.drop([19730, 29503, 35587])
metadata['year'] = pd.to_datetime(metadata['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

metadata['id'] = metadata['id'].astype('int')
smd = metadata[metadata['id'].isin(links)]

smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

/var/folders/62/_jg5mmj53wn7r5q71_chkt440000gn/T/ipykernel_61370/1681699468.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = metadata.apply(lambda x: pd.Series(x['genres']), axis=1).stack().reset_index(level=1, drop=True)
/var/folders/62/_jg5mmj53wn7r5q71_chkt440000gn/T/ipykernel_61370/1681699468.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['tagline'] = smd['tagline'].fillna('')
/var/folders/62/_jg5mmj53wn7r5q71_chkt440000gn/T/ipykernel_61370/1681699468.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [4]:
credits = pd.read_csv('dataset/credits.csv')
keywords = pd.read_csv('dataset/keywords.csv')

keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
metadata['id'] = metadata['id'].astype('int')

metadata = metadata.merge(credits, on='id')
metadata = metadata.merge(keywords, on='id')

smd = metadata[metadata['id'].isin(links)]

smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

/var/folders/62/_jg5mmj53wn7r5q71_chkt440000gn/T/ipykernel_61370/1196386596.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['cast'] = smd['cast'].apply(literal_eval)
/var/folders/62/_jg5mmj53wn7r5q71_chkt440000gn/T/ipykernel_61370/1196386596.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['crew'] = smd['crew'].apply(literal_eval)
/var/folders/62/_jg5mmj53wn7r5q71_chkt440000gn/T/ipykernel_61370/1196386596.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [5]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

smd['director'] = smd['crew'].apply(get_director)
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x,x, x])

/var/folders/62/_jg5mmj53wn7r5q71_chkt440000gn/T/ipykernel_61370/667881306.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['director'] = smd['crew'].apply(get_director)
/var/folders/62/_jg5mmj53wn7r5q71_chkt440000gn/T/ipykernel_61370/667881306.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
/var/folders/62/_jg5mmj53wn7r5q71_chkt440000gn/T/ipykernel_61370/667881306.py:9: SettingWithCopyWarning: 
A val

In [6]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

/var/folders/62/_jg5mmj53wn7r5q71_chkt440000gn/T/ipykernel_61370/848702291.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['keywords'] = smd['keywords'].apply(filter_keywords)
/var/folders/62/_jg5mmj53wn7r5q71_chkt440000gn/T/ipykernel_61370/848702291.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
/var/folders/62/_jg5mmj53wn7r5q71_chkt440000gn/T/ipykernel_61370/848702291.py:10: SettingWithCopyWarning: 
A value is try

In [7]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])
cosine_sim = cosine_similarity(count_matrix, count_matrix)
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [8]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

get_recommendations('The Dark Knight').head(10)

8031      The Dark Knight Rises
6218              Batman Begins
6623               The Prestige
2085                  Following
4145                   Insomnia
3381                    Memento
8613               Interstellar
7648                  Inception
5943                   Thursday
8927    Kidnapping Mr. Heineken
Name: title, dtype: object

In [9]:
reader = Reader()
ratings = pd.read_csv('dataset/ratings_small.csv')
ratings.head()
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=10, verbose=True)
trainset = data.build_full_trainset()
svd.fit(trainset)

Evaluating RMSE, MAE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.8938  0.8928  0.8900  0.8882  0.8960  0.8866  0.8949  0.8890  0.8786  0.9058  0.8916  0.0067  
MAE (testset)     0.6854  0.6844  0.6849  0.6867  0.6901  0.6814  0.6844  0.6866  0.6812  0.6976  0.6863  0.0045  
Fit time          6.03    5.86    5.91    5.39    5.35    5.34    5.94    5.98    5.75    5.37    5.69    0.28    
Test time         0.06    0.08    0.15    0.06    0.06    0.14    0.07    0.06    0.07    0.06    0.08    0.03    


In [10]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

id_map = pd.read_csv('dataset/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')
indices_map = id_map.set_index('id')

In [11]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    movie_id = id_map.loc[title]['movieId']

    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:30]
    movie_indices = [i[0] for i in sim_scores]

    movies = smd.iloc[movie_indices][['title', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    del movies['est']
    return movies.head(10).to_json(orient = 'records')

In [12]:
hybrid(16, 'The Martian')

'[{"title":"Interstellar","id":157336},{"title":"Blade Runner","id":78},{"title":"Black Hawk Down","id":855},{"title":"Gladiator","id":98},{"title":"Alien","id":348},{"title":"Jurassic Park","id":329},{"title":"Matchstick Men","id":7270},{"title":"Body of Lies","id":12113},{"title":"Stargate: Continuum","id":12914},{"title":"Someone to Watch Over Me","id":31650}]'

In [13]:
import pickle

with open('data.pickle', 'wb') as f:
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

In [14]:
with open('svd.pickle', 'wb') as f:
    pickle.dump(svd, f, pickle.HIGHEST_PROTOCOL)

In [15]:
with open('dataset.pickle', 'wb') as f:
    pickle.dump(smd, f, pickle.HIGHEST_PROTOCOL)

In [16]:
with open('linkMap.pickle', 'wb') as f:
    pickle.dump(indices_map, f, pickle.HIGHEST_PROTOCOL)

In [17]:
smd['title'][0]

'Toy Story'

In [18]:
'Toy Story' in smd.values

True